In [ ]:

from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）
DATASET_SLUG = 'gns-codes'
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 展開済み code/ を優先し、無ければ code.zip を展開
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code も code.zip もありません。Add Data で {DATASET_SLUG} を追加してください。")

# フラット化（zip解凍で1階層挟まった場合）
if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children) == 1 and children[0].is_dir():
        inner = children[0]
        for p in inner.iterdir():
            p.rename(repo_dir / p.name)
        inner.rmdir()

%cd $repo_dir


In [ ]:

# PyG の radius_graph で必要になる torch-cluster だけインストール
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda
cuda_tag = 'cpu' if cuda_ver is None else 'cu' + cuda_ver.replace('.', '')
url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print('PyG wheels:', url)

!pip -q install torch-cluster -f {url}
!pip -q install torch_geometric


In [ ]:

from pathlib import Path
import copy
import yaml
import re
import json

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

# モデルごとに対応するデータセットを自動解決する

def _load_metadata(path: Path):
    try:
        with path.open('r', encoding='utf-8') as f:
            return json.load(f)
    except Exception:
        return None

def _find_metadata(root: Path):
    candidates = [
        root / 'metadata.json',
        root / 'metadata' / 'metadata.json',
    ]
    candidates += list(root.glob('**/metadata*.json'))
    for c in candidates:
        if c.is_file():
            meta = _load_metadata(c)
            if isinstance(meta, dict):
                return meta, c
    return None, None

def _extract_sequence_length(meta: dict):
    if not isinstance(meta, dict):
        return None
    for key in ('train', 'valid', 'test', 'rollout'):
        part = meta.get(key)
        if isinstance(part, dict) and 'sequence_length' in part:
            try:
                return int(part['sequence_length'])
            except Exception:
                pass
    if 'sequence_length' in meta:
        try:
            return int(meta['sequence_length'])
        except Exception:
            return None
    return None

def resolve_dataset_root(step: int | None):
    base = Path('/kaggle/input')
    if not base.exists():
        return None, False

    # 1) ディレクトリ名にステップ数が含まれるものを優先
    if step is not None:
        for dataset_root in sorted(base.iterdir()):
            name = dataset_root.name
            if (name.endswith(f'-{step}') or name.endswith(f'_{step}')
                    or f'-{step}-' in name or f'_{step}_' in name):
                return dataset_root, False

    # 2) metadata の sequence_length で一致を探す
    for dataset_root in sorted(base.iterdir()):
        meta, meta_path = _find_metadata(dataset_root)
        if not meta:
            continue
        seq = _extract_sequence_length(meta)
        if step is not None and seq == step:
            return dataset_root, False

    # 3) フォールバック（もっとも汎用な800データセット）
    fallback = base / 'dam-break-left-800'
    if fallback.exists():
        print(f"[fallback] step={step} に一致するデータセットが見つからず、{fallback} を使用します。")
        return fallback, True
    print(f"[fallback] step={step} に一致するデータセットも既定の dam-break-left-800 も見つかりません。")
    return None, True

# 出力設定（共通）
output_root = REPO / 'rollouts'
viz_format = 'html'  # html|mp4|gif

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

# データセットはモデルごとに後続セルで差し替える
cfg['method'] = 'gns'
cfg['rollout_inference_max_examples'] = None
cfg['output_path'] = str(output_root)
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = '/kaggle/input'  # placeholder

# モデルは後続セルで差し替える（ベースを保存しておく）
cfg['model_path'] = str(REPO / 'models')
cfg['model_file'] = None
cfg['output_filename'] = 'rollout'

BASE_CFG = copy.deepcopy(cfg)

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)

print('ベース設定を書き出しました:', cfg_path)
print('output_path:', cfg['output_path'])



In [ ]:

%cd /kaggle/working/code

import copy
import re
import subprocess
from pathlib import Path
import numpy as np

from analyze_rollouts import analyze_rollout

# 比較対象モデルを集約（rollout_diff 優先）
model_roots = [
    REPO / 'models' / 'rollout_diff',
    REPO / 'models',
]
model_files: list[Path] = []
for root in model_roots:
    if not root.exists():
        continue
    for path in root.glob('model-*.pt'):
        model_files.append(path.resolve())

# 重複除去 & ソート（親ディレクトリ優先→ステップ番号）
seen = set()
unique_models: list[Path] = []
for p in model_files:
    if p in seen:
        continue
    seen.add(p)
    unique_models.append(p)

def _step_key(p: Path):
    m = re.search(r'(\d+)', p.stem)
    step = int(m.group(1)) if m else -1
    priority = 0 if p.parent.name == 'rollout_diff' else 1
    return (priority, p.parent.name, step)

def _step_from_path(p: Path):
    m = re.search(r'(\d+)(?!.*\d)', p.stem)
    return int(m.group(1)) if m else None

model_files = sorted(unique_models, key=_step_key)

if not model_files:
    raise FileNotFoundError('model-*.pt が見つかりません。models/ 下に配置してください。')

print('評価対象モデル:')
for p in model_files:
    print(' -', p)

results = []

cmd = ['python', 'src/train.py', '--config', str(cfg_path)]

for model_path in model_files:
    tag = f"{model_path.parent.name}-{model_path.stem}"
    step_num = _step_from_path(model_path)
    dataset_root, used_fallback = resolve_dataset_root(step_num)
    if dataset_root is None:
        raise FileNotFoundError(f"対応するデータセットが見つかりません（step={step_num}）")

    cfg = copy.deepcopy(BASE_CFG)
    cfg['model_path'] = str(model_path.parent)
    cfg['model_file'] = str(model_path)
    cfg['output_filename'] = f"rollout_{tag}"
    cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = str(dataset_root)
    cfg['data_path'] = str(dataset_root)
    cfg['rollout_dataset'] = 'valid'

    with cfg_path.open('w', encoding='utf-8') as f:
        yaml.safe_dump(cfg, f, allow_unicode=True)

    print(f"\n=== {tag} ===")
    print('dataset:', dataset_root)
    if used_fallback:
        print('[warn] 指定ステップに一致するデータセットが見つからなかったためフォールバックを使用しました。')
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

    output_dir = Path(cfg['output_path']) / cfg['method'] / cfg['output_filename']
    pkl_files = sorted(output_dir.glob(f"{cfg['output_filename']}_ex*.pkl"))
    if not pkl_files:
        raise FileNotFoundError(f"{output_dir} に *_ex*.pkl がありません。推論が成功したか確認してください。")

    dist_list = []
    for pkl_path in pkl_files:
        res_one = analyze_rollout(pkl_path)
        dist_list.append(res_one['distance_error_per_timestep'])

    min_len = min(len(a) for a in dist_list)
    dist_stack = np.stack([a[:min_len] for a in dist_list])
    avg_dist = dist_stack.mean(axis=0)
    mean_full = float(avg_dist.mean())

    results.append({
        'tag': tag,
        'distance_error_per_timestep': avg_dist.tolist(),
        'mean_distance_error': mean_full,
        'n_rollouts': len(dist_list),
        'used_length': min_len,
        'pkl_dir': str(output_dir),
        'dataset_root': str(dataset_root),
        'step_num': step_num,
        'used_fallback': used_fallback,
    })

    print(f"  rollouts: {len(dist_list)} / used length: {min_len} / mean distance error: {mean_full:.6f}")



In [ ]:

import re
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

MAX_SUMMARY_STEPS = 100
# None なら上限なし（従来は 0 で無表示になっていた）
MAX_PLOT_STEPS = None  # 1ステップ誤差可視化の表示上限
# モデルごとに評価を打ち切る最大ステップ数を指定（なければ全ステップ）
STEP_LIMITS = {
    'rollout_diff-model-100': 100,
    'rollout_diff-model-200': 200,
    'rollout_diff-model-400': 400,
    'rollout_diff-model-800': 800,
}
AUTO_LIMIT_FROM_TAG = True  # タグ末尾の数値 (例: model-400) を上限として使う

def resolve_limit(tag: str, total_len: int) -> int:
    if tag in STEP_LIMITS:
        return min(STEP_LIMITS[tag], total_len)
    if AUTO_LIMIT_FROM_TAG:
        m = re.search(r'(\d+)(?!.*\d)', tag)
        if m:
            return min(int(m.group(1)), total_len)
    return total_len

if not results:
    print('results が空です。前のセルを実行してください。')
else:
    # -------- 全タイムステップの距離誤差（従来のラインプロット） --------
    plt.figure(figsize=(10, 6))
    for r in results:
        err = np.array(r['distance_error_per_timestep'])
        limit = resolve_limit(r['tag'], len(err))
        err = err[:limit]
        timesteps = np.arange(len(err))
        plt.plot(
            timesteps,
            err,
            label=f"{r['tag']} (T={len(err)})",
        )
    plt.xlabel('timestep')
    plt.ylabel('mean distance error')
    plt.title('Rollout distance error per timestep')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()

    plot_path = output_root / 'distance_error_comparison.png'
    plt.savefig(plot_path, dpi=150)
    plt.show()
    print('プロットを保存:', plot_path)

    # -------- 1ステップ誤差（タイムステップごと、散布＋線） --------
    plt.figure(figsize=(10, 6))
    for r in results:
        err = np.array(r['distance_error_per_timestep'])
        limit = resolve_limit(r['tag'], len(err))
        if MAX_PLOT_STEPS is not None:
            limit = min(limit, MAX_PLOT_STEPS)
        timesteps = np.arange(limit)
        plt.plot(timesteps, err[:limit], marker='o', markersize=3, linewidth=1.2,
                 label=f"{r['tag']} (T={limit})")
    plt.xlabel('timestep')
    plt.ylabel('mean distance error (1-step)')
    max_label = 'all' if MAX_PLOT_STEPS is None else MAX_PLOT_STEPS
    plt.title(f'1-step error per timestep (up to {max_label} or tag limit)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()

    plot_path_step = output_root / 'distance_error_per_timestep.png'
    plt.savefig(plot_path_step, dpi=150)
    plt.show()
    print('1ステップ誤差プロットを保存:', plot_path_step)

    # -------- サマリ --------
    print('\nサマリ（平均距離誤差と長さ）')
    for r in results:
        err = np.array(r['distance_error_per_timestep'])
        limit = resolve_limit(r['tag'], len(err))
        err = err[:limit]
        mean_full = float(err.mean())
        mean_100 = float(err[:min(MAX_SUMMARY_STEPS, len(err))].mean())
        print(f"- {r['tag']}: steps={len(err)}, mean_full={mean_full:.6f}, mean@<=100={mean_100:.6f}")

